#Boston 311 Tutorial

This notebook will run you through the basic usage of this package to train 3 models on the Boston 311 mydata and use them to predict the outcome of cases from the last 30 days

In [1]:
! pip install ../

Defaulting to user installation because normal site-packages is not writeable


Processing /home/briarmoss/Documents/Boston_311
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for boston311: filename=boston311-0.1.0-py3-none-any.whl size=19337 sha256=739437fbefd0f43b2a1d03652e9a75e7cca5c9ebcbf8e2bc4bb960ba8a979409
  Stored in directory: /tmp/pip-ephem-wheel-cache-_j_u4mnp/wheels/3d/69/ee/0a6ac96b9c09c948fc0e74f2724a9703aa39749a41fa757c9e
Successfully built boston311
  Attempting uninstall: boston311
    Found existing installation: boston311 0.1.0
    Uninstalling boston311-0.1.0:
      Successfully uninstalled boston311-0.1.0


##Import the Boston311Model class

In [2]:
import os
import pandas as pd
import numpy as np
import pickle
import re
import sys
import time

In [3]:
from boston311 import Boston311KerasNLP


2023-10-27 21:47:46.411282: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-27 21:47:46.629238: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-27 21:47:46.630104: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-27 21:47:47.356595: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Using TensorFlow backend


## Load extra features

In [4]:
#get current datetime in Boston timezone as string
from datetime import datetime
from pytz import timezone
import pytz
boston = timezone('US/Eastern')
now = datetime.now(boston)
today_datestring = now.strftime("%Y-%m-%d")
#get time in Boston timezone as string for a filename
now = datetime.now(boston)
time_string = now.strftime("%H-%M-%S")
#define datetime string
my_datetime = today_datestring + '_' + time_string 

#format tomorrows date as yyyy-mm-dd
tomorrows_date =  now + pd.DateOffset(days=1)
tomorrows_datestring = tomorrows_date.strftime("%Y-%m-%d")

In [5]:
print('today_datestring: ', today_datestring)
print('tomorrows_datestring: ', tomorrows_datestring)

today_datestring:  2023-10-27
tomorrows_datestring:  2023-10-28


In [6]:
#set path to mydata
#EXTRA_mydata_FILE = './cls_and_pooled_embeddings_with_service_id.csv'
EXTRA_mydata_FILE = './cls_and_pooled_embeddings_with_three_cols.csv'


In [7]:
import glob
import os

def get_latest_model_files(model_dir="./daily_models/Boston311KerasNLP/"):
    # Get all json and h5 files
    json_files = glob.glob(os.path.join(model_dir, "*.json"))
    h5_files = glob.glob(os.path.join(model_dir, "*.h5"))

    # Sort files by modification time
    json_files.sort(key=os.path.getmtime, reverse=True)
    h5_files.sort(key=os.path.getmtime, reverse=True)

    if json_files and h5_files:
        latest_json = json_files[0]
        latest_h5 = h5_files[0]
        return latest_json, latest_h5
    else:
        return None, None

# Use the function
json_file, model_file = get_latest_model_files()
print(json_file)

if json_file and model_file:
    kerasNLP_model = Boston311KerasNLP()
    kerasNLP_model.load(json_file, model_file)
    kerasNLP_model.predict_date_range['end'] = tomorrows_datestring


./daily_models/Boston311KerasNLP/20231027_214725_Boston311KerasNLP.json


In [8]:
#load data from all_311_data.csv
mydata = pd.read_csv('all_311_cases.csv', low_memory=False)

In [9]:
data = kerasNLP_model.load_data( data=mydata, train_or_predict='predict' )
data = kerasNLP_model.enhance_data( data, 'predict')

/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['closed_dt'] = pd.to_datetime(data['closed_dt'])
/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['open_dt'] = pd.to_datetime(data['open_dt'])
/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [10]:
cyclical_df = data[['case_enquiry_id', 'season_cos', 'weekday_cos']].copy()

In [11]:
#show first ten records with newest open_dt
data.sort_values(by=['open_dt'], ascending=False).head(10)

,case_enquiry_id,open_dt,sla_target_dt,closed_dt,on_time,case_status,closure_reason,case_title,subject,reason,...,latitude,longitude,geom_4326,source,survival_time,event,ward_number,season_cos,weekday_cos,survival_time_hours
2733348,101005143479,2023-10-26 20:02:54,NaN,NaT,ONTIME,Open,,Schedule Bulk Item Pickup,Public Works Department,Sanitation,...,42.350600,-71.160581,0101000020E610000061183FF446CA51C08B1A1173E02C...,Self Service,NaT,0,22,0.402214,-0.900969,NaN
2733347,101005143476,2023-10-26 19:46:27,2023-10-30 04:30:00,NaT,ONTIME,Open,,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,42.337423,-71.029280,0101000020E61000006B19B1BADFC151C05660B2AE302B...,Citizens Connect App,NaT,0,6,0.402214,-0.900969,NaN
2733346,101005143470,2023-10-26 19:06:20,2023-10-30 04:30:00,NaT,ONTIME,Open,,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,42.351234,-71.064129,0101000020E6100000CB6E7BB01AC451C0FDFB3640F52C...,Citizens Connect App,NaT,0,3,0.402214,-0.900969,NaN
2733345,101005143469,2023-10-26 19:04:37,2023-10-30 04:30:00,NaT,ONTIME,Open,,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,42.350570,-71.063610,0101000020E6100000504FAC3112C451C009505876DF2C...,Citizens Connect App,NaT,0,3,0.402214,-0.900969,NaN
2733344,101005143468,2023-10-26 19:03:10,2023-10-30 04:30:00,NaT,ONTIME,Open,,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,42.351170,-71.063590,0101000020E61000008AF6C7DD11C451C0C0DC811FF32C...,Citizens Connect App,NaT,0,3,0.402214,-0.900969,NaN
2733343,101005143467,2023-10-26 18:55:31,2023-10-30 04:30:00,NaT,ONTIME,Open,,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,42.375149,-71.033340,0101000020E610000061F6933F22C251C0E2CFFCE50430...,Citizens Connect App,NaT,0,1,0.402214,-0.900969,NaN
2733342,101005143466,2023-10-26 18:54:53,2023-11-13 03:30:00,NaT,ONTIME,Open,,Street Light Outages,Public Works Department,Street Lights,...,42.276290,-71.069452,0101000020E61000000570B1E771C451C02FA4627A5D23...,Citizens Connect App,NaT,0,17,0.402214,-0.900969,NaN
2733341,101005143464,2023-10-26 18:48:25,2023-10-30 04:30:00,NaT,ONTIME,Open,,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,42.328210,-71.093521,0101000020E6100000392DCC3DFCC551C057B36BC5022A...,Citizens Connect App,NaT,0,9,0.402214,-0.900969,NaN
2733340,101005143463,2023-10-26 18:48:03,2024-02-23 17:48:04,NaT,ONTIME,Open,,Pest Infestation - Residential,Inspectional Services,Housing,...,42.259883,-71.117177,0101000020E6100000928A7AD27FC751C019AE68D84321...,Citizens Connect App,NaT,0,18,0.402214,-0.900969,NaN
2733339,101005143462,2023-10-26 18:44:13,2023-10-30 04:30:00,NaT,ONTIME,Open,,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,42.333849,-71.033962,0101000020E6100000018AD3702CC251C035FE7F94BB2A...,Citizens Connect App,NaT,0,6,0.402214,-0.900969,NaN


In [12]:
clean_data = kerasNLP_model.clean_data_for_prediction( data )

In [13]:
clean_data.head()

,case_enquiry_id,queue_ANML02_LostFound,queue_ANML_General,queue_BHA_General,queue_BPD_Administrative,queue_BPD_Districts,queue_BPS_Administrative,queue_BPS_Transportation Administration,queue_BTDT_AVRS Interface Queue,queue_BTDT_Abandoned Bicycle,...,department_ECON,department_GEN_,department_GRNi,department_INFO,department_ISD,department_No Q,department_ONS_,department_PARK,department_PROP,department_PWDx
0,101005082967,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,101005082968,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,101005082970,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
3,101005082972,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
4,101005082974,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [14]:
mydata = clean_data

In [15]:
mydata['case_enquiry_id']

0        101005082967
1        101005082968
2        101005082970
3        101005082972
4        101005082974
             ...     
24883    101005143468
24884    101005143469
24885    101005143470
24886    101005143476
24887    101005143479
Name: case_enquiry_id, Length: 24888, dtype: object

In [16]:
import pandas as pd
import numpy as np
from ast import literal_eval
import pickle

pickle_file = 'dataframe.pkl'

X = None

if os.path.exists(pickle_file):
    #check if the file date is earlier than EXTRA_mydata_FILE date
    pickle_file_date = os.path.getmtime(pickle_file)
    EXTRA_mydata_FILE_date = os.path.getmtime(EXTRA_mydata_FILE)
    if pickle_file_date < EXTRA_mydata_FILE_date:
        os.remove(pickle_file)

if os.path.exists(pickle_file):

    X = pickle.load(open(pickle_file, "rb"))
else:
    X = pd.read_csv(EXTRA_mydata_FILE)

    #rename service_request_id to case_enquiry_id
    X.rename(columns={'service_request_id':'case_enquiry_id'}, inplace=True)
    #remove all rows where case_enquiry_id is non-numeric
    #X = X[X['case_enquiry_id'].str.isnumeric()]
    #convert case_enquiry_id to int64
    #X['case_enquiry_id'] = X['case_enquiry_id'].astype('int64')

    # Convert stringified arrays back to NumPy arrays
    for col in ['desc_cls_embedding', 'desc_pooled_embedding', 'name_cls_embedding', 'name_pooled_embedding', 'code_cls_embedding', 'code_pooled_embedding']:
        X[col] = X[col].apply(literal_eval).apply(np.array)

    pickle.dump(X, open(pickle_file, "wb"))



In [17]:
#concatenate the two dataframes and reindex
df = X

In [18]:
df.shape

(291559, 7)

In [19]:


# Assuming df is your DataFrame and it has columns 'desc_cls_embedding', 'desc_pooled_embedding', 'name_cls_embedding', 'name_pooled_embedding', 'code_cls_embedding', 'code_pooled_embedding'
desc_cls_embedding_flattened = np.stack(df['desc_cls_embedding'].to_numpy())
desc_pooled_embedding_flattened = np.stack(df['desc_pooled_embedding'].to_numpy())
#do the same for the rest
name_cls_embedding_flattened = np.stack(df['name_cls_embedding'].to_numpy())
name_pooled_embedding_flattened = np.stack(df['name_pooled_embedding'].to_numpy())
code_cls_embedding_flattened = np.stack(df['code_cls_embedding'].to_numpy())
code_pooled_embedding_flattened = np.stack(df['code_pooled_embedding'].to_numpy())


# Remove the old columns
df.drop(['desc_cls_embedding', 'desc_pooled_embedding'], axis=1, inplace=True)
#do the same for the rest
df.drop(['name_cls_embedding', 'name_pooled_embedding'], axis=1, inplace=True)
df.drop(['code_cls_embedding', 'code_pooled_embedding'], axis=1, inplace=True)


# Add the new flattened columns
df_desc_cls = pd.DataFrame(desc_cls_embedding_flattened, columns=[f'desc_cls_{i}' for i in range(desc_cls_embedding_flattened.shape[1])])

df_desc_pooled = pd.DataFrame(desc_pooled_embedding_flattened, columns=[f'desc_pooled_{i}' for i in range(desc_pooled_embedding_flattened.shape[1])])
#do the same for the rest
df_name_cls = pd.DataFrame(name_cls_embedding_flattened, columns=[f'name_cls_{i}' for i in range(name_cls_embedding_flattened.shape[1])])
df_name_pooled = pd.DataFrame(name_pooled_embedding_flattened, columns=[f'name_pooled_{i}' for i in range(name_pooled_embedding_flattened.shape[1])])
df_code_cls = pd.DataFrame(code_cls_embedding_flattened, columns=[f'code_cls_{i}' for i in range(code_cls_embedding_flattened.shape[1])])
df_code_pooled = pd.DataFrame(code_pooled_embedding_flattened, columns=[f'code_pooled_{i}' for i in range(code_pooled_embedding_flattened.shape[1])])



df = pd.concat([df, df_desc_cls, df_desc_pooled, df_name_cls, df_name_pooled, df_code_cls, df_code_pooled], axis=1)

In [20]:
df['case_enquiry_id'] = df['case_enquiry_id'].astype(str)
is_numeric = df['case_enquiry_id'].str.isnumeric()

In [21]:
df = df[is_numeric]

In [22]:
df['case_enquiry_id'] = df['case_enquiry_id'].astype('int64')

In [23]:
df.shape

(291559, 769)

In [24]:
mydata.shape

(24888, 253)

In [25]:
#join them so we are left only with records that have mydata in both files
new_mydata = mydata.merge(df, on='case_enquiry_id', how='inner')


In [26]:
new_mydata = new_mydata.merge(cyclical_df, on='case_enquiry_id', how='inner')

In [27]:
new_mydata.shape

(16238, 1023)

In [28]:

df = new_mydata

In [29]:
#cast all columns that are type bool to float
for col in df.columns:
    if df[col].dtype == 'bool':
        df[col] = df[col].astype('float64')
    if df[col].dtype == 'int64':
        df[col] = df[col].astype('float64') 

In [30]:
#list the number of rows in X and y
print(df.dtypes)


case_enquiry_id              object
queue_ANML02_LostFound      float64
queue_ANML_General          float64
queue_BHA_General           float64
queue_BPD_Administrative    float64
                             ...   
code_pooled_125             float64
code_pooled_126             float64
code_pooled_127             float64
season_cos                  float64
weekday_cos                 float64
Length: 1023, dtype: object


In [31]:
#free all unused dataframes
#df_to_delete = [cls_embedding_flattened, pooled_embedding_flattened, df_cls, df_pooled, X, new_mydata, is_numeric, mydata]

#for data_frame in df_to_delete:
#    if data_frame is not None:
#        del data_frame

In [32]:
case_enquiry_id = df['case_enquiry_id']
X_predict = df.drop(['case_enquiry_id'], axis=1)


In [33]:

#parse CLS embedding column as array
predictions = kerasNLP_model.model.predict(X_predict)

508/508 [==============================] - 2s 3ms/step


In [34]:
# Define a function to flatten an array into a string.
def array_to_string(arr):
    return ' '.join(map(str, arr))

# Apply the function along axis 1 (rows).
string_predictions = np.apply_along_axis(array_to_string, axis=1, arr=predictions)

# Now string_predictions is a 1D NumPy array where each element is a string
# that contains all the elements from the corresponding row in the original 2D array.
print(string_predictions)

['0.14967792 0.08314113 0.018538207 0.010064403 0.0114389155 0.003023907 0.005235482 0.0022375751 0.0033338084 0.0043094778 0.0033349262 0.0044662496 0.006847348 0.004615168 0.013201882 0.004159459 0.015149556 0.0035336183 0.005651924 0.005801963 0.0033331297 0.008101259 0.0023807052 0.009372704 0.0025525265 0.0051987786 0.004689568 0.0019937896 0.0053906306 0.0009981843 0.00589795 0.0014599708 0.0027097624 0.0017524345 0.0013912526 0.004759935 0.00077795266 0.0048408704 0.0012589595 0.0034707855 0.0020251796 0.001134797 0.005631366 0.00088897656 0.0058931476 0.0008737036 0.0031234268 0.0030627938 0.0013058949 0.005489373 0.0006022524 0.0056985454 0.0016736478 0.0037482786 0.0043130633 0.0024033554 0.0098742945 0.0015002568 0.00679286 0.0015687449 0.502302'
 '0.8624625 0.11623729 0.00810726 0.0032283207 0.0027713263 0.0007885699 0.0010340562 0.0002998415 0.0008632109 0.0009552336 0.00061705697 0.0008972857 0.0005399899 0.00033795362 6.870146e-05 3.4944205e-05 5.5225606e-05 3.496864e-05

In [35]:
#combine case_enquiry_id and predictions into a dataframe
predictions_df = pd.DataFrame({'case_enquiry_id':case_enquiry_id, 'prediction':string_predictions})

In [36]:
bin_labels = [
    "0-12 hours",      # Less than half a day
    "12-24 hours",     # Half to one day
    "1-3 days",        # One to three days
    "4-7 days",        # Four to seven days
    "1-2 weeks",       # One to two weeks
    "2-4 weeks",       # Two to four weeks
    "1-2 months",      # One to two months
    "2-4 months",      # Two to four months
    "4+ months"        # More than four months
]

#predictions_df['prediction'] = predictions_df['prediction'].apply(lambda x: bin_labels[x])

In [37]:
len(predictions_df)

16238

In [38]:
print(predictions)

[[1.4967792e-01 8.3141133e-02 1.8538207e-02 ... 6.7928601e-03
  1.5687449e-03 5.0230199e-01]
 [8.6246252e-01 1.1623729e-01 8.1072599e-03 ... 1.6325919e-05
  2.2271568e-06 2.1122796e-05]
 [9.9555105e-01 2.6412667e-03 4.4293868e-04 ... 4.2291285e-06
  7.1808358e-06 2.4399364e-04]
 ...
 [8.1761783e-01 1.7020674e-01 4.6038353e-03 ... 2.0298501e-05
  1.2731821e-06 3.8850678e-05]
 [8.1857497e-01 1.6864941e-01 4.5213546e-03 ... 2.6664318e-05
  1.6243853e-06 4.7649432e-05]
 [8.2504362e-01 1.6456516e-01 3.9704400e-03 ... 1.6687321e-05
  9.6449264e-07 3.3644570e-05]]


In [39]:
kerasNLP_model.model_type

'Boston311KerasNLP'

In [40]:
import pandas as pd

#get model_name from json_file name and ml_model_date from json_file name first 8 characters which are YYYYMMDD and change it to YYYY-MM-DD
model_name = json_file.split('/')[-1].split('.')[0]
ml_model_date = model_name[:4] + '-' + model_name[4:6] + '-' + model_name[6:8]

#define an empt pandas dataframe ml_model_df
ml_model_df = pd.DataFrame(columns=['ml_model_name', 'ml_model_type', 'ml_model_date'])

ml_model_df = pd.concat([ml_model_df, pd.DataFrame([{'ml_model_name': model_name, 
                                    'ml_model_type': kerasNLP_model.model_type,
                                    'ml_model_date': ml_model_date}])], ignore_index=True)

print(ml_model_df)

                       ml_model_name      ml_model_type ml_model_date
0  20231027_214725_Boston311KerasNLP  Boston311KerasNLP    2023-10-27


In [41]:
model_cases = data.drop(['geom_4326','survival_time_hours', 'survival_time', 'event','season_cos', 'weekday_cos'], axis=1).copy()

In [42]:
all_model_cases = model_cases 
all_model_predictions = predictions_df

In [43]:
all_model_predictions['ml_model_name'] = model_name

In [44]:
all_model_predictions['prediction_date'] = today_datestring

In [45]:
# %%
#get current datetime in Boston timezone as string
from datetime import datetime
from pytz import timezone
import pytz
boston = timezone('US/Eastern')
now = datetime.now(boston)
today_datestring = now.strftime("%Y-%m-%d")
#get time in Boston timezone as string for a filename
now = datetime.now(boston)
time_string = now.strftime("%H-%M-%S")
#define datetime string
my_datetime = today_datestring + '_' + time_string 

In [46]:


# %%
all_model_cases.to_csv(my_datetime+'_311_cases.csv', index=False)


# %%

all_model_predictions.to_csv(my_datetime+'_311_predictions.csv', index=False)

# %%

ml_model_df.to_csv(my_datetime+'_311_ml_models.csv', index=False)

# %%
#create datetime _manifest.txt file with one filename per line
with open(my_datetime+'_manifest.txt', 'w') as f:
    f.write(my_datetime+'_311_cases.csv\n')
    f.write(my_datetime+'_311_predictions.csv\n')
    f.write(my_datetime+'_311_ml_models.csv\n')

# %%
#create an export folder
EXPORT_FOLDER = '~/Documents/BODC-DEI-site/database/seeders'
#copy the csv files to the export folder
!cp {my_datetime}_311_cases.csv {EXPORT_FOLDER}
!cp {my_datetime}_311_predictions.csv {EXPORT_FOLDER}
!cp {my_datetime}_311_ml_models.csv {EXPORT_FOLDER}
!cp {my_datetime}_manifest.txt {EXPORT_FOLDER}



# %% [markdown]
# ** Copy the files to the production server **

# %%
# Define constants for servers
PROD_USER = 'u353344964'
PROD_HOSTNAME = '195.179.236.61'
PORT_NUMBER = 65002
PROD_BASE_FOLDER = '/home/u353344964/domains/bodc-dei.org/laravel'
STAGE_BASE_FOLDER = '/home/u353344964/domains/bodc-dei.org/stagelaravel'
PROD_EXPORT_FOLDER = '/home/u353344964/domains/bodc-dei.org/laravel/database/seeders'
STAGE_EXPORT_FOLDER = '/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders'

# %%







In [47]:
import os


def scp_to_server(filename, user=PROD_USER, hostname=PROD_HOSTNAME, port=PORT_NUMBER, export_folder=PROD_EXPORT_FOLDER):
    """Copy a file to the server using scp."""
    command = f"scp -P {port} {filename} {user}@{hostname}:{export_folder}"
    print(f"Executing: {command}")
    os.system(command)

# Use the function to scp files
files_to_copy = [
    f"{my_datetime}_311_cases.csv",
    f"{my_datetime}_311_predictions.csv",
    f"{my_datetime}_311_ml_models.csv",
    f"{my_datetime}_manifest.txt"
]

# Control where to copy
copy_to_prod = True
copy_to_stage = True

for file in files_to_copy:
    if copy_to_prod:
        scp_to_server(file, export_folder=PROD_EXPORT_FOLDER)
    if copy_to_stage:
        scp_to_server(file, export_folder=STAGE_EXPORT_FOLDER)


# %%
PORT_NUMBER

Executing: scp -P 65002 2023-10-27_21-48-45_311_cases.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/laravel/database/seeders


Executing: scp -P 65002 2023-10-27_21-48-45_311_cases.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders
Executing: scp -P 65002 2023-10-27_21-48-45_311_predictions.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/laravel/database/seeders
Executing: scp -P 65002 2023-10-27_21-48-45_311_predictions.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders
Executing: scp -P 65002 2023-10-27_21-48-45_311_ml_models.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/laravel/database/seeders
Executing: scp -P 65002 2023-10-27_21-48-45_311_ml_models.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders
Executing: scp -P 65002 2023-10-27_21-48-45_manifest.txt u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/laravel/database/seeders
Executing: scp -P 65002 2023-10-27_21-48-45_manifest.txt u353344964@195.179.236.61:/

65002

In [48]:

if copy_to_prod:
    ! ssh -p {PORT_NUMBER} {PROD_USER}@{PROD_HOSTNAME} 'cd {PROD_BASE_FOLDER}; php artisan db:seed --class=ThreeOneOneSeeder'

if copy_to_stage:
    ! ssh -p {PORT_NUMBER} {PROD_USER}@{PROD_HOSTNAME} 'cd {STAGE_BASE_FOLDER}; php artisan db:seed --class=ThreeOneOneSeeder'


   INFO  Seeding database.  


Manifest files:
/home/u353344964/domains/bodc-dei.org/laravel/database/seeders/2023-10-27_21-48-45_manifest.txt

Total records to process: 41127

Processing /home/u353344964/domains/bodc-dei.org/laravel/database/seeders/2023-10-27_21-48-45_311_cases.csv
100 App\Models\ThreeOneOneCase records processed.
Records remaining in this file: 24788.
Total records remaining: 41027.
Time for last 100 records: 0.02 seconds.
Estimated time remaining for this file: 5 seconds.
Estimated time for all files: 9 seconds.
200 App\Models\ThreeOneOneCase records processed.
Records remaining in this file: 24688.
Total records remaining: 40927.
Time for last 100 records: 0.01 seconds.
Estimated time remaining for this file: 2 seconds.
Estimated time for all files: 3 seconds.
300 App\Models\ThreeOneOneCase records processed.
Records remaining in this file: 24588.
Total records remaining: 40827.
Time for last 100 records: 0.01 seconds.
Estimated time remaining for this file: 1 se